## Exploring the City of Saint Louis public crime data with Apache Spark 2.3

The Saint Louis OpenData project contains hundreds of datasets for the city of Saint Louis. Open government data has the potential to increase the quality of life for residents, create more efficient government services, better public decisions, and even create new local businesses and services.

![Gateway Arch](https://images.unsplash.com/photo-1514893011-72dfa15bd29c?ixlib=rb-0.3.5&ixid=eyJhcHBfaWQiOjEyMDd9&s=8c2d25b2fcf02c87b1e9022f43affd05&auto=format&fit=crop&w=1491&q=80)

We start by mounting the Amazon S3 storage to the notebook.

In [5]:
ACCESS_KEY = "XXXXXXXXXXXXXXX"
SECRET_KEY = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
ENCODED_SECRET_KEY = SECRET_KEY.replace("/", "%2F")
AWS_BUCKET_NAME = "xxxxx-xxxx"
MOUNT_NAME = "input"

dbutils.fs.mount("s3a://%s:%s@%s" % (ACCESS_KEY, ENCODED_SECRET_KEY, AWS_BUCKET_NAME), "/mnt/%s" % MOUNT_NAME)
display(dbutils.fs.ls("/mnt/%s" % MOUNT_NAME))

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-4495755652863896> in <module> () 
 7 MOUNT_NAME = "input" 
 8 
 ----> 9 dbutils . fs . mount ( "s3a://%s:%s@%s" % ( ACCESS_KEY , ENCODED_SECRET_KEY , AWS_BUCKET_NAME ) , "/mnt/%s" % MOUNT_NAME ) 
 10 display ( dbutils . fs . ls ( "/mnt/%s" % MOUNT_NAME ) ) 

 /tmp/1545402801002-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 227 class ExecutionError ( Exception ) : 
 228 pass 
 --> 229 raise ExecutionError ( str ( e ) ) 
 230 return f_with_exception_handling 
 231 

 ExecutionError : An error occurred while calling o192.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/input; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/input
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:101)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:55)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:394)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/input
	at scala.Predef$.require(Predef.scala:277)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:127)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:66)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:83)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:82)
	at scala.collection.immutable.List.foreach(List.scala:388)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:82)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:272)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:252)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:42)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:58)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:58)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:38)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$2(UsageLogging.scala:359)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:235)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:58)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:230)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:227)
	at com.databricks.rpc.ServerBackend.withAttributionContext(ServerBackend.scala:13)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:268)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:264)
	at com.databricks.rpc.ServerBackend.withA

List all mounted points

In [7]:
display(dbutils.fs.mounts())

mountPoint,source,encryptionType
/mnt/input,s3a://boong-data,
/mnt//mnt/sf_open_data,s3a://databricks-corp-training/sf_open_data/,
/,DatabricksRoot,sse-s3
/databricks-datasets,databricks-datasets,sse-s3
/databricks-results,databricks-results,sse-s3


The 2018 crime data of Saint Louis city is uploaded to S3 in csv format and pulled down to Databricks for analysis. You can list the file with the `%fs ls` command

In [9]:
%fs ls /mnt/input/input

path,name,size
dbfs:/mnt/input/input/2018stlcrimedata.csv,2018stlcrimedata.csv,6901629


Note, I combined 12 csv files into 1 file to get a full year of data.  I downloaded directly from this link: http://www.slmpd.org/Crimereports.shtml

I'm using "spark" as an entry point into all functionality in Spark 2.3.

In [12]:
spark

Out[ 1 ]: <pyspark.sql.session.SparkSession at 0x7fb3af0d2b38>

Using the SparkSession, create a DataFrame from the CSV file by inferring the schema.

In [14]:
crimeDataDF = spark.read.csv('/mnt/input/input/2018stlcrimedata.csv', header=True, inferSchema=True)

In [15]:
crimeDataDF.count()

Out[ 3 ]: 46742

Display the data using a display function by Databricks.

In [17]:
display(crimeDataDF)

Complaint,CodedMonth,DateOccur,FlagCrime,FlagUnfounded,FlagAdministrative,Count,FlagCleanup,Crime,District,Description,ILEADSAddress,ILEADSStreet,Neighborhood,LocationName,LocationComment,CADAddress,CADStreet,XCoord,YCoord
18-001804,2018-01,1/1/2018 0:01,Y,,,1,,31111,1,ROBBERY-HIGHWAY /FIREARM USED/SUCCESSFUL,6431,IDAHO AVE,1,,ON STREET IN FRONT OF 6431 IDAHO,6431,IDAHO,890047.4,992586.3
18-002209,2018-01,1/1/2018 0:01,Y,,,1,,142320,5,DESTRUCTION OF PROPERTY-MALICIOUS/PRIV PROP,5506,ETZEL AVE,48,,,5506,ETZEL,884087.5,1030426.0
18-000063,2018-01,1/1/2018 0:01,Y,,,1,,67601,4,"LARCENY-FROM BUILDING $500 - $24,999",2543,N GRAND BLVD,59,THE BOULEVARD,,2543,GRAND,899361.5,1026965.0
18-000018,2018-01,1/1/2018 0:01,Y,,,1,,41011,2,AGG.ASSAULT-FIREARM/CITIZEN ADULT 1ST DEGREE,4000,DE TONTY ST,27,,,4000,DE TONTY,892323.3,1013550.0
18-000003,2018-01,1/1/2018 0:01,Y,,,1,,41011,3,AGG.ASSAULT-FIREARM/CITIZEN ADULT 1ST DEGREE,3149,PESTALOZZI ST,25,,,3149,PESTALOZZI,896039.9,1008823.0
18-000132,2018-01,1/1/2018 0:01,Y,,,1,,142320,6,DESTRUCTION OF PROPERTY-MALICIOUS/PRIV PROP,4901,HOOKE AVE,71,,,4901,HOOKE,894124.1,1038665.0
18-000010,2018-01,1/1/2018 0:01,Y,,,1,,41021,5,AGG.ASSAULT-FIREARM/CITIZEN CHILD 1ST DEGREE,4711,WASHINGTON AVE,38,,SINGLE FAMILY HOME,4711,WASHINGTON,889258.8,1025742.0
18-000010,2018-01,1/1/2018 0:01,Y,,,1,,41011,5,AGG.ASSAULT-FIREARM/CITIZEN ADULT 1ST DEGREE,4711,WASHINGTON AVE,38,,SINGLE FAMILY HOME,4711,WASHINGTON,889258.8,1025742.0
18-003411,2018-01,1/1/2018 0:01,Y,,,1,,142320,4,DESTRUCTION OF PROPERTY-MALICIOUS/PRIV PROP,1405,PINE ST,36,,APARTMENT BUILDING,1405,PINE,905730.9,1018275.0
18-005259,2018-01,1/1/2018 0:01,Y,,,1,,67701,1,LARCENY-FROM BUILDING UNDER $500,4201,BEETHOVEN AVE,5,,,1015,PARK,887948.4,1002640.0


Notice that the above cell takes ~2 seconds to run b/c it is inferring the schema by sampling the file and reading through it.

Inferring the schema works for ad-hoc analysis against smaller datasets. But when working on terabyte of data, it's better to provide an **explicit pre-defined schema manually**, so there's no inferring cost:

In [19]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, DecimalType

Note that we are removing all space characters from the col names to prevent errors when writing to Parquet later

In [21]:
crimeSchema = StructType(
                     [
                     StructField('Complaint', StringType(), True),
                     StructField('CodedMonth', StringType(), True),
                     StructField('DateOccur', StringType(), True),
                     StructField('FlagCrime', StringType(), True),                  
                     StructField('FlagUnfounded', StringType(), True),       
                     StructField('FlagAdministrative', StringType(), True),       
                     StructField('Count', IntegerType(), True),       
                     StructField('FlagCleanup', StringType(), True),       
                     StructField('Crime', IntegerType(), True),       
                     StructField('District', IntegerType(), True),       
                     StructField('Description', StringType(), True),       
                     StructField('ILEADSAddress', IntegerType(), True),                  
                     StructField('ILEADSStreet', StringType(), True),       
                     StructField('Neighborhood', IntegerType(), True),       
                     StructField('LocationName', StringType(), True),       
                     StructField('LocationComment', StringType(), True),       
                     StructField('CADAddress', IntegerType(), True),       
                     StructField('CADStreet', StringType(), True),       
                     StructField('XCoord', StringType(), True),                 
                     StructField('YCoord', StringType(), True)
                     ]
                     )

In [22]:
crimeDataSDF = spark.read.csv('/mnt/input/input/2018stlcrimedata.csv', header=True, schema=crimeSchema)

In [23]:
display(crimeDataSDF)

Complaint,CodedMonth,DateOccur,FlagCrime,FlagUnfounded,FlagAdministrative,Count,FlagCleanup,Crime,District,Description,ILEADSAddress,ILEADSStreet,Neighborhood,LocationName,LocationComment,CADAddress,CADStreet,XCoord,YCoord
18-001804,2018-01,1/1/2018 0:01,Y,,,1,,31111,1,ROBBERY-HIGHWAY /FIREARM USED/SUCCESSFUL,6431,IDAHO AVE,1,,ON STREET IN FRONT OF 6431 IDAHO,6431,IDAHO,890047.4,992586.3
18-002209,2018-01,1/1/2018 0:01,Y,,,1,,142320,5,DESTRUCTION OF PROPERTY-MALICIOUS/PRIV PROP,5506,ETZEL AVE,48,,,5506,ETZEL,884087.5,1030426
18-000063,2018-01,1/1/2018 0:01,Y,,,1,,67601,4,"LARCENY-FROM BUILDING $500 - $24,999",2543,N GRAND BLVD,59,THE BOULEVARD,,2543,GRAND,899361.5,1026965
18-000018,2018-01,1/1/2018 0:01,Y,,,1,,41011,2,AGG.ASSAULT-FIREARM/CITIZEN ADULT 1ST DEGREE,4000,DE TONTY ST,27,,,4000,DE TONTY,892323.3,1013550
18-000003,2018-01,1/1/2018 0:01,Y,,,1,,41011,3,AGG.ASSAULT-FIREARM/CITIZEN ADULT 1ST DEGREE,3149,PESTALOZZI ST,25,,,3149,PESTALOZZI,896039.9,1008823
18-000132,2018-01,1/1/2018 0:01,Y,,,1,,142320,6,DESTRUCTION OF PROPERTY-MALICIOUS/PRIV PROP,4901,HOOKE AVE,71,,,4901,HOOKE,894124.1,1038665
18-000010,2018-01,1/1/2018 0:01,Y,,,1,,41021,5,AGG.ASSAULT-FIREARM/CITIZEN CHILD 1ST DEGREE,4711,WASHINGTON AVE,38,,SINGLE FAMILY HOME,4711,WASHINGTON,889258.8,1025742
18-000010,2018-01,1/1/2018 0:01,Y,,,1,,41011,5,AGG.ASSAULT-FIREARM/CITIZEN ADULT 1ST DEGREE,4711,WASHINGTON AVE,38,,SINGLE FAMILY HOME,4711,WASHINGTON,889258.8,1025742
18-003411,2018-01,1/1/2018 0:01,Y,,,1,,142320,4,DESTRUCTION OF PROPERTY-MALICIOUS/PRIV PROP,1405,PINE ST,36,,APARTMENT BUILDING,1405,PINE,905730.9,1018275
18-005259,2018-01,1/1/2018 0:01,Y,,,1,,67701,1,LARCENY-FROM BUILDING UNDER $500,4201,BEETHOVEN AVE,5,,,1015,PARK,887948.4,1002640


The csv file contains null records so we drop all null records from the table.

In [25]:
crimeDataSDF.na.drop(subset="Complaint")

Out[ 10 ]: DataFrame[Complaint: string, CodedMonth: string, DateOccur: string, FlagCrime: string, FlagUnfounded: string, FlagAdministrative: string, Count: int, FlagCleanup: string, Crime: int, District: int, Description: string, ILEADSAddress: int, ILEADSStreet: string, Neighborhood: int, LocationName: string, LocationComment: string, CADAddress: int, CADStreet: string, XCoord: string, YCoord: string]

In [26]:
crimeDataSDF.count()

Out[ 12 ]: 46742

Look at the first 5 records in the DataFrame:

In [28]:
crimeDataSDF.show(5)

+---------+----------+-------------+---------+-------------+------------------+-----+-----------+------+--------+--------------------+-------------+-------------+------------+-------------+--------------------+----------+----------+--------+--------+
Complaint|CodedMonth| DateOccur|FlagCrime|FlagUnfounded|FlagAdministrative|Count|FlagCleanup| Crime|District| Description|ILEADSAddress| ILEADSStreet|Neighborhood| LocationName| LocationComment|CADAddress| CADStreet| XCoord| YCoord|
+---------+----------+-------------+---------+-------------+------------------+-----+-----------+------+--------+--------------------+-------------+-------------+------------+-------------+--------------------+----------+----------+--------+--------+
18-001804| 2018-01|1/1/2018 0:01| Y| | | 1| | 31111| 1|ROBBERY-HIGHWAY ...| 6431| IDAHO AVE| 1| |ON STREET IN FRON...| 6431| IDAHO|890047.4|992586.3|
18-002209| 2018-01|1/1/2018 0:01| Y| | | 1| |142320| 5|DESTRUCTION OF PR...| 5506| ETZEL AVE| 48| | | 5506| ETZEL|884087.5| 1030426|
18-000063| 2018-01|1/1/2018 0:01| Y| | | 1| | 67601| 4|LARCENY-FROM BUIL...| 2543| N GRAND BLVD| 59|THE BOULEVARD| | 2543| GRAND|899361.5| 1026965|
18-000018| 2018-01|1/1/2018 0:01| Y| | | 1| | 41011| 2|AGG.ASSAULT-FIREA...| 4000| DE TONTY ST| 27| | | 4000| DE TONTY|892323.3| 1013550|
18-000003| 2018-01|1/1/2018 0:01| Y| | | 1| | 41011| 3|AGG.ASSAULT-FIREA...| 3149|PESTALOZZI ST| 25| | | 3149|PESTALOZZI|896039.9| 1008823|
+---------+----------+-------------+---------+-------------+------------------+-----+-----------+------+--------+--------------------+-------------+-------------+------------+-------------+--------------------+----------+----------+--------+--------+
only showing top 5 rows

Print just the column names in the DataFrame:

In [30]:
crimeDataSDF.columns

Out[ 14 ]: 
['Complaint',
 'CodedMonth',
 'DateOccur',
 'FlagCrime',
 'FlagUnfounded',
 'FlagAdministrative',
 'Count',
 'FlagCleanup',
 'Crime',
 'District',
 'Description',
 'ILEADSAddress',
 'ILEADSStreet',
 'Neighborhood',
 'LocationName',
 'LocationComment',
 'CADAddress',
 'CADStreet',
 'XCoord',
 'YCoord']

Count how many rows total there are in DataFrame (and see how long it takes to do a full scan from remote disk/S3):

In [32]:
crimeDataSDF.count()

Out[ 16 ]: 46742

There are over ~46 thousand rows in the DataFrame and it takes ~2 seconds to do a full read of it.

###  **Analysis with PySpark DataFrames and Spark SQL API**

Create a temp view to use spark.sql

In [36]:
crimeDataSDF.createOrReplaceTempView("crimesql")

**Q-1) How many different types of calls were made to the Police Department?**

In [38]:
crimeDataSDF.select('Description').distinct().show(35, False)

+----------------------------------------------+
Description |
+----------------------------------------------+
BURGLARY-BUSINESS/DAY/UNLAW ENT/OCCUPIED |
DRUGS-POSSESSION/PENTAZOCINE |
ROBBERY-GAS STA/KNIFE USED/SUCCESSFUL |
ASSLT-AGGRV-KNIFE/CUT-1ST-ADULT-DOMESTIC |
AGG.ASSAULT-OTH DANG WEP/CTZEN CHLD 2ND DEGREE|
ASSLT-AGGRAV-OTH-WPN-3RD-ADULT-DOMESTIC |
ROBBERY-RESIDENCE /KNIFE USED/SUCCESSFUL |
AGG.ASSAULT-FIREARM/CITIZEN ADULT 1ST DEGREE |
AGG.ASSAULT-HNDS,FST,FEET/CTZEN CHLD 1ST DEGRE|
SEX OFFNS-SX MISCONDUCT/SUS<17&VIC 14-15/NO FO|
BURGLARY-RESIDENCE/NIT/ATT FORCE ENTRY |
DESTRUCTION OF PROPERTY-MALICIOUS/PUBLIC PROP |
LARCENY-FROM COIN MACH UNDER $500 |
ROBBERY CARJACKING /STRNGARM/NO INJ/SUCCESSFUL|
ASSLT-AGGRV-KNIFE/CUT-2ND-CHILD-DOMESTIC |
STLG BY DECEIT/IDENTITY THEFT REPORT |
FRAUD-CREDIT DEVICE/UNDER $150 |
OBSTRUCT GOVRN OP-FALSE REPORT |
AGG.ASSAULT-KNIFE/CITIZEN CHILD 3RD DEGREE |
DISORDERLY CONDUCT-PEACE DSTRB/SCHOOL |
AGG.ASSAULT-FIREARM/POLC.OFFICER 3RD DEGREE |
SEX OFFNS-EXPOSURE/LEWD&LACIV ACT |
LEAVING SCENE OF ACCIDENT |
AGG.ASSAULT-KNIFE/CITIZEN CHILD 1ST DEGREE |
FRAUD-DEFRAUDING SECURED CREDITOR |
FAMILY & CHILD CRIME-CNTRIB DELINQ MINOR |
BURGLARY-RESIDENCE/NIT/UNLAW ENT/OCCUPIED |
ASSLT-AGGRAV-FIREARM-3RD-CHILD-DOMESTIC |
LARCENY-FROM MTR VEH UNDER $500 /ATTEMPT |
ASSLT-AGGRV-HND/FST/FT-2ND-ADUL-DOMESTIC |
LARCENY-ALL OTHER OVER $25,000 |
ROBBERY-COMMERCE PL/STRNGARM/NO INJ/SUCCESS |
OBSTRUCT GOVRN OP-JUDIC/TMPR WTH EVDNCE |
RAPE -- INCEST (FORCIBLE) |
FRAUD-STEALING BY DECEIT/UNDER $150 |
+----------------------------------------------+
only showing top 35 rows

In [39]:
display(sqlContext.sql("SELECT Description FROM crimesql GROUP BY description"), limit=35)

Description
BURGLARY-BUSINESS/DAY/UNLAW ENT/OCCUPIED
DRUGS-POSSESSION/PENTAZOCINE
ROBBERY-GAS STA/KNIFE USED/SUCCESSFUL
ASSLT-AGGRV-KNIFE/CUT-1ST-ADULT-DOMESTIC
AGG.ASSAULT-OTH DANG WEP/CTZEN CHLD 2ND DEGREE
ASSLT-AGGRAV-OTH-WPN-3RD-ADULT-DOMESTIC
ROBBERY-RESIDENCE /KNIFE USED/SUCCESSFUL
AGG.ASSAULT-FIREARM/CITIZEN ADULT 1ST DEGREE
"AGG.ASSAULT-HNDS,FST,FEET/CTZEN CHLD 1ST DEGRE"
SEX OFFNS-SX MISCONDUCT/SUS<17&VIC 14-15/NO FO


The queries above show the different type of calls to the police department.

**Q-2) How many incidents of each call type were there?**

In [42]:
display(crimeDataSDF.select('Description').groupBy('Description').count().orderBy('Count', ascending=False))

Description,count
LEAVING SCENE OF ACCIDENT,5505
DESTRUCTION OF PROPERTY-MALICIOUS/PRIV PROP,3658
LARCENY-FROM MTR VEH UNDER $500,2285
LARCENY-MTR VEH PARTS UNDER $500,2026
AUTO THEFT-PERM RETNT/UNRECOV OVER 48HR,1873
"LARCENY-FROM MTR VEH $500 - $24,999",1566
PUBLIC ORDER-OTHR UNSPC PBLC ORDER VIOLATION,1530
SIMPLE ASSAULT-ADULT/NO INJURY,1418
AGG.ASSAULT-FIREARM/CITIZEN ADULT 1ST DEGREE,1325
LARCENY-FROM BUILDING UNDER $500,1305


In [43]:
display(sqlContext.sql("SELECT Description, count(*) as count FROM crimesql GROUP BY description ORDER BY count desc"))

Description,count
LEAVING SCENE OF ACCIDENT,5505
DESTRUCTION OF PROPERTY-MALICIOUS/PRIV PROP,3658
LARCENY-FROM MTR VEH UNDER $500,2285
LARCENY-MTR VEH PARTS UNDER $500,2026
AUTO THEFT-PERM RETNT/UNRECOV OVER 48HR,1873
"LARCENY-FROM MTR VEH $500 - $24,999",1566
PUBLIC ORDER-OTHR UNSPC PBLC ORDER VIOLATION,1530
SIMPLE ASSAULT-ADULT/NO INJURY,1418
AGG.ASSAULT-FIREARM/CITIZEN ADULT 1ST DEGREE,1325
LARCENY-FROM BUILDING UNDER $500,1305


Seems like the Saint Louis City Police department is called for leaving crime scene far more than any other type. Note that the above command took about 3 seconds to execute. In an upcoming section, we'll cache the data into memory for up to 100x speed increases.

**Q-4) What is the most dangerous month in Saint Louis city?**

In [46]:
display(crimeDataSDF.select('CodedMonth').groupBy('CodedMonth').count().orderBy('Count',ascending=False))

CodedMonth,count
2018-08,4402
2018-06,4282
2018-07,4257
2018-09,4096
2018-10,4087
2018-05,4013
2018-01,3825
2018-04,3735
2018-12,3672
2018-03,3629


In [47]:
display(sqlContext.sql("select CodedMonth, count(*) as count from crimesql group by CodedMonth ORDER BY count DESC"))

CodedMonth,count
2018-08,4402
2018-06,4282
2018-07,4257
2018-09,4096
2018-10,4087
2018-05,4013
2018-01,3825
2018-04,3735
2018-12,3672
2018-03,3629


Seems like August 2018 is the most dangerous month in Saint Louis city and June comes in second.

### ![Spark Logo Tiny](http://curriculum-release.s3-website-us-west-2.amazonaws.com/wiki-book/general/logo_spark_tiny.png) ** Doing Date/Time Analysis**

**Q-4) How many service calls were logged on July 4th?**

Notice that the date or time columns (DateOccur) is currently being interpreted as strings, rather than date or time objects:

In [52]:
crimeDataSDF.printSchema()

root
-- Complaint: string (nullable = true)
-- CodedMonth: string (nullable = true)
-- DateOccur: string (nullable = true)
-- FlagCrime: string (nullable = true)
-- FlagUnfounded: string (nullable = true)
-- FlagAdministrative: string (nullable = true)
-- Count: integer (nullable = true)
-- FlagCleanup: string (nullable = true)
-- Crime: integer (nullable = true)
-- District: integer (nullable = true)
-- Description: string (nullable = true)
-- ILEADSAddress: integer (nullable = true)
-- ILEADSStreet: string (nullable = true)
-- Neighborhood: integer (nullable = true)
-- LocationName: string (nullable = true)
-- LocationComment: string (nullable = true)
-- CADAddress: integer (nullable = true)
-- CADStreet: string (nullable = true)
-- XCoord: string (nullable = true)
-- YCoord: string (nullable = true)

Let's use the unix_timestamp() function to convert the string into a timestamp:

https://people.apache.org/~pwendell/spark-nightly/spark-master-docs/latest/api/python/pyspark.sql.html?highlight=spark#pyspark.sql.functions.from_unixtime

In [54]:
from pyspark.sql.functions import *

In [55]:
# Note that PySpark uses the Java Simple Date Format patterns

from_pattern1 = 'MM/dd/yyyy'
to_pattern1 = 'MM/dd/yyyy'

from_pattern2 = 'MM/dd/yyyy HH:mm'
to_pattern2 = 'MM/dd/yyyy HH:mm'


crimeDataSTsDF = crimeDataSDF \
  .withColumn('DateOccurTS', unix_timestamp(crimeDataSDF['DateOccur'], from_pattern2).cast("timestamp")) \
  .drop('DateOccur')

In [56]:
crimeDataSTsDF.printSchema()

root
-- Complaint: string (nullable = true)
-- CodedMonth: string (nullable = true)
-- FlagCrime: string (nullable = true)
-- FlagUnfounded: string (nullable = true)
-- FlagAdministrative: string (nullable = true)
-- Count: integer (nullable = true)
-- FlagCleanup: string (nullable = true)
-- Crime: integer (nullable = true)
-- District: integer (nullable = true)
-- Description: string (nullable = true)
-- ILEADSAddress: integer (nullable = true)
-- ILEADSStreet: string (nullable = true)
-- Neighborhood: integer (nullable = true)
-- LocationName: string (nullable = true)
-- LocationComment: string (nullable = true)
-- CADAddress: integer (nullable = true)
-- CADStreet: string (nullable = true)
-- XCoord: string (nullable = true)
-- YCoord: string (nullable = true)
-- DateOccurTS: timestamp (nullable = true)

In [57]:
crimeDataSTsDF.createOrReplaceTempView("crimesql_ts")

Notice that the formatting of the timestamps is now different:

Note that July 4th, is the 185th day of the year in 2018.

Filter the DF down to just 2018 and days of year equal 185:

In [60]:
crimeDataSTsDF.filter(year('DateOccurTs') == '2018').filter(dayofyear('DateOccurTs') == 185).groupBy(dayofyear('DateOccurTs')).count().orderBy(dayofyear('DateOccurTs')).show()

+----------------------+-----+
dayofyear(DateOccurTs)|count|
+----------------------+-----+
 185| 133|
+----------------------+-----+

In [61]:
display(sqlContext.sql("select count(*) as count FROM crimesql_ts WHERE cast(DateOccurTs as date)='2018-07-04'"))

count
133


In [62]:
They were 133 calls made on July 4, 2018.

**Q-5) How many service calls were logged in the first week of January 2018?**

Note that we can narrow down to the 2018 year and look at the first 7 days.

In [65]:
crimeDataSTsDF.filter(year('DateOccurTs') == '2018').filter(dayofyear('DateOccurTs') >= 1).filter(dayofyear('DateOccurTs') <= 7).groupBy(dayofyear('DateOccurTs')).count().orderBy(dayofyear('DateOccurTs')).show()

+----------------------+-----+
dayofyear(DateOccurTs)|count|
+----------------------+-----+
 1| 185|
 2| 96|
 3| 138|
 4| 98|
 5| 139|
 6| 114|
 7| 100|
+----------------------+-----+

In [66]:
display(sqlContext.sql("SELECT cast(DateOccurTs as date) as Date, count(*) as count FROM crimesql_ts Where cast(DateOccurTs as date) between '2018-01-01' and '2018-01-07' Group BY cast(DateOccurTs as date) ORDER BY Date "))

Date,count
2018-01-01,185
2018-01-02,96
2018-01-03,138
2018-01-04,98
2018-01-05,139
2018-01-06,114
2018-01-07,100


Visualize the results in a bar graph:

In [68]:
display(crimeDataSTsDF.filter(year('DateOccurTs') == '2018').filter(dayofyear('DateOccurTs') >= 1).filter(dayofyear('DateOccurTs') <= 7).groupBy(dayofyear('DateOccurTs')).count().orderBy(dayofyear('DateOccurTs')))

dayofyear(DateOccurTs),count
1,183
2,95
3,138
4,98
5,139
6,114
7,100


### ![Spark Logo Tiny](http://curriculum-release.s3-website-us-west-2.amazonaws.com/wiki-book/general/logo_spark_tiny.png) ** Memory, Caching and write to Parquet**

The DataFrame is currently comprised of 2 partitions:

In [71]:
crimeDataSTsDF.rdd.getNumPartitions()

Out[ 87 ]: 2

the repartition to 3 so that the data is divided evenly among 3 slots on Databrick Community Edition.

In [73]:
crimeDataSTsDF.repartition(3).createOrReplaceTempView("crimeDataVIEW");

In [74]:
spark.catalog.cacheTable("crimeDataVIEW")

In [75]:
# Call .count() to materialize the cache
spark.table("crimeDataVIEW").count()

Out[ 91 ]: 43070

In [76]:
crimeDataDF = spark.table("crimeDataVIEW")

Once the data is cached, the full table scan from Amazon S3 took 1/10 of a second verus 2 seconds as before.

In [78]:
crimeDataDF.count()

Out[ 93 ]: 43070

In [79]:
spark.catalog.isCached("crimeDataVIEW")

Out[ 94 ]: True

The 3 partitions are now cached in memory to match the Databrick 3 slots.

![6 Partitions](http://curriculum-release.s3-website-us-west-2.amazonaws.com/sf_open_data_meetup/df_6_parts.png)

We can check the Spark UI to see the 3 partitions in memory:

Now that our data has the correct date/time types for each column and it is correctly partitioned, let's write it down as a parquet file for future loading:

In [84]:
%fs ls /mnt/input/input

path,name,size
dbfs:/mnt/input/input/2018stlcrimedata.csv,2018stlcrimedata.csv,6901629


In [85]:
crimeDataDF.write.format('parquet').save('dbfs:/mnt/input/input/data')

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-3000173949531881> in <module> () 
 ----> 1 crimeDataDF . write . format ( 'parquet' ) . save ( 'dbfs:/mnt/input/input' ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 714 self . _jwrite . save ( ) 
 715 else : 
 --> 716 self . _jwrite . save ( path ) 
 717 
 718 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 67 e.java_exception.getStackTrace()))
 68 if s . startswith ( 'org.apache.spark.sql.AnalysisException: ' ) : 
 ---> 69 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 70 if s . startswith ( 'org.apache.spark.sql.catalyst.analysis' ) : 
 71 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 

 AnalysisException : 'path dbfs:/mnt/input/input already exists.;'

Now the directory should contain 3 .gz compressed Parquet files (one for each partition):

In [87]:
%fs ls dbfs:/mnt/input/input/data

path,name,size
dbfs:/mnt/input/input/data/_SUCCESS,_SUCCESS,0
dbfs:/mnt/input/input/data/_committed_8939891421050246980,_committed_8939891421050246980,321
dbfs:/mnt/input/input/data/_started_8939891421050246980,_started_8939891421050246980,0
dbfs:/mnt/input/input/data/part-00000-tid-8939891421050246980-6fd6f2b4-0502-41d7-a4eb-d84f94bd19d0-4214-c000.snappy.parquet,part-00000-tid-8939891421050246980-6fd6f2b4-0502-41d7-a4eb-d84f94bd19d0-4214-c000.snappy.parquet,462381
dbfs:/mnt/input/input/data/part-00001-tid-8939891421050246980-6fd6f2b4-0502-41d7-a4eb-d84f94bd19d0-4215-c000.snappy.parquet,part-00001-tid-8939891421050246980-6fd6f2b4-0502-41d7-a4eb-d84f94bd19d0-4215-c000.snappy.parquet,463558
dbfs:/mnt/input/input/data/part-00002-tid-8939891421050246980-6fd6f2b4-0502-41d7-a4eb-d84f94bd19d0-4216-c000.snappy.parquet,part-00002-tid-8939891421050246980-6fd6f2b4-0502-41d7-a4eb-d84f94bd19d0-4216-c000.snappy.parquet,461637


Here's how you can easily read the parquet file from S3 in the future:

In [89]:
tempDF = spark.read.parquet('dbfs:/mnt/input/input/data/')

In [90]:
display(tempDF.limit(2))

Complaint,CodedMonth,FlagCrime,FlagUnfounded,FlagAdministrative,Count,FlagCleanup,Crime,District,Description,ILEADSAddress,ILEADSStreet,Neighborhood,LocationName,LocationComment,CADAddress,CADStreet,XCoord,YCoord,DateOccurTS
18-029558,2018-06,Y,,,1,,64701,4,LARCENY-FROM MTR VEH UNDER $500,421,S 10TH ST,35,CUPPLES STATION GARAGE,,1000,CLARK,906905.9,1016099,2018-06-27T09:00:00.000+0000
18-021971,2018-05,Y,,,1,,265321,6,LEAVING SCENE OF ACCIDENT,0,SWITZER AVE / TRAFFORD LN,73,,,1827,ETON,891600.6,1049902,2018-05-16T21:55:00.000+0000


The possibilities are endless with these datasets.

In [92]:
This notebook was inspired by Sameer Farooqui at Databricks.